In [1]:
!pip install indic-nlp-library

Defaulting to user installation because normal site-packages is not writeable
  Using cached indic_nlp_library-0.92-py3-none-any.whl.metadata (5.7 kB)
  Using cached Morfessor-2.0.6-py3-none-any.whl.metadata (628 bytes)
  Using cached sphinxcontrib_jquery-4.1-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached sphinxcontrib_jsmath-1.0.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached snowballstemmer-2.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached imagesize-1.4.1-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached indic_nlp_library-0.92-py3-none-any.whl (40 kB)
Using cached Morfessor-2.0.6-py3-none-any.whl (35 kB)
   ---------------------------------------- 0.0/7.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.7 MB ? eta -:--:--
   -------- ------------------------------- 1.6/7.7 MB 7.0 MB/s eta 0:00:01
   -------------------- ------------------- 3.9/7.7 MB 9.4 MB/s eta 0:00:01
   --------------------- ------------------ 4.2/7.7 MB 6.6 MB/s eta 0

DEPRECATION: Loading egg at c:\program files\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [8]:
import nltk
import re
import unicodedata
from nltk.corpus import stopwords
from indicnlp.tokenize import indic_tokenize, sentence_tokenize


In [4]:
# Function to normalize text
def normalize_text(text):
    # Unicode normalization
    text = unicodedata.normalize('NFC', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters, numbers, and retain only Nepali characters
    text = re.sub(r'[^\u0900-\u097F\s]', ' ', text)
    
    # Remove single Nepali characters
    text = re.sub(r'\s+[\u0900-\u097F]\s+', ' ', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text 

In [11]:
nepali_stopwords = set([
    'अक्सर', 'अगाडि', 'अझै', 'अनुसार', 'अन्तर्गत', 'अन्य', 'अन्यत्र', 'अन्यथा', 'अब', 'अरू', 'अरूलाई', 
    'अर्को', 'अर्थात', 'अर्थात्', 'अलग', 'आए', 'आजको', 'आठ', 'आत्म', 'आदि', 'आफू', 'आफूलाई', 'आफैलाई', 
    'आफ्नै', 'आफ्नो', 'आयो', 'उदाहरण', 'उन', 'उनको', 'उनले', 'उप', 'उहाँलाई', 'एउटै', 'एक', 'एकदम', 
    'औं', 'कतै', 'कम से कम', 'कसरी', 'कसै', 'कसैले', 'कहाँबाट', 'कहिलेकाहीं', 'कहिल्यै', 'कहीं', 
    'का', 'कि', 'किन', 'किनभने', 'कुनै', 'कुरा', 'कृपया', 'के', 'केहि', 'केही', 'को', 'कोही', 'क्रमशः', 
    'गए', 'गरि', 'गरी', 'गरेका', 'गरेको', 'गरेर', 'गरौं', 'गर्छ', 'गर्छु', 'गर्दै', 'गर्न', 'गर्नु', 
    'गर्नुपर्छ', 'गर्ने', 'गर्यौं', 'गैर', 'चाँडै', 'चार', 'चाले', 'चाहनुहुन्छ', 'चाहन्छु', 'चाहिए', 'छ', 
    'छन्', 'छु', 'छैन', 'छौँ', 'छौं', 'जताततै', 'जब', 'जबकि', 'जसको', 'जसबाट', 'जसमा', 'जसलाई', 'जसले', 
    'जस्तै', 'जस्तो', 'जस्तोसुकै', 'जहाँ', 'जान', 'जाहिर', 'जुन', 'जे', 'जो', 'ठीक', 'त', 'तत्काल', 
    'तथा', 'तदनुसार', 'तपाइँको', 'तपाईं', 'तर', 'तल', 'तापनि', 'तिनी', 'तिनीहरू', 'तिनीहरूको', 'तिनीहरूलाई', 
    'तिनीहरूले', 'तिमी', 'तिर', 'ती', 'तीन', 'तुरुन्तै', 'तेस्रो', 'त्यसकारण', 'त्यसपछि', 'त्यसमा', 'त्यसैले', 
    'त्यहाँ', 'त्यो', 'थिए', 'थिएन', 'थिएनन्', 'थियो', 'दिए', 'दिनुभएको', 'दिनुहुन्छ', 'दुई', 'देख', 'देखि', 
    'देखिन्छ', 'देखियो', 'देखे', 'देखेको', 'देखेर', 'देख्न', 'दोश्रो', 'दोस्रो', 'धेरै', 'न', 'नजिकै', 'नत्र', 
    'नयाँ', 'नि', 'निम्ति', 'निम्न', 'निम्नानुसार', 'निर्दिष्ट', 'नै', 'नौ', 'पक्का', 'पक्कै', 'पछि', 
    'पछिल्लो', 'पटक', 'पनि', 'पर्छ', 'पर्थ्यो', 'पर्याप्त', 'पहिले', 'पहिलो', 'पहिल्यै', 'पाँच', 'पाँचौं', 
    'पूर्व', 'प्रति', 'प्रत्येक', 'प्लस', 'फेरि', 'बने', 'बन्द', 'बन्न', 'बरु', 'बाटो', 'बारे', 'बाहिर', 
    'बाहेक', 'बीच', 'बीचमा', 'भए', 'भएको', 'भन', 'भने', 'भने्', 'भन्छन्', 'भन्छु', 'भन्दा', 'भन्नुभयो', 
    'भन्ने', 'भर', 'भित्र', 'भित्री', 'म', 'मलाई', 'मा', 'मात्र', 'माथि', 'मुख्य', 'मेरो', 'यति', 
    'यथोचित', 'यदि', 'यद्यपि', 'यस', 'यसको', 'यसपछि', 'यसबाहेक', 'यसरी', 'यसो', 'यस्तो', 'यहाँ', 
    'यहाँसम्म', 'या', 'यी', 'यो', 'र', 'रही', 'रहेका', 'रहेको', 'राखे', 'राख्छ', 'राम्रो', 'रूप', 
    'लगभग', 'लाई', 'लागि', 'ले', 'वरिपरि', 'वास्तवमा', 'वाहेक', 'विरुद्ध', 'विशेष', 'शायद', 'सँग', 
    'सँगै', 'सक्छ', 'सट्टा', 'सधैं', 'सबै', 'सबैलाई', 'समय', 'सम्भव', 'सम्म', 'सही', 'साँच्चै', 
    'सात', 'साथ', 'साथै', 'सायद', 'सारा', 'सो', 'सोध्न', 'सोही', 'स्पष्ट', 'हरे', 'हरेक', 'हामी', 
    'हामीलाई', 'हाम्रो', 'हुँ', 'हुन', 'हुने', 'हुनेछ', 'हुन्', 'हुन्छ', 'हो', 'होइन', 'होइनन्', 'होला', 
    'होस्'
])


In [5]:
def remove_stopwords(text):
    stop_words = set(stopwords.words('nepali'))
    words = text.split()
    return ' '.join([word for word in words if word not in stop_words])

In [6]:
def preprocessing(text):
    # Normalize text
    text = normalize_text(text)
    
    # Tokenize words
    tokens = list(indic_tokenize.trivial_tokenize(text, lang='ne'))
    
    # Remove stopwords
    text = remove_stopwords('text')
    
    # Return preprocessed text
    return text

In [13]:
text = "नेपालको राजधानी काठमाण्डौ हो। यसमा विभिन्न ठाउँका मानिसहरू बस्छन्।"
preprocessed_text = preprocessing(text)
print(preprocessed_text)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\L E G I O N/nltk_data'
    - 'c:\\Program Files\\Python312\\nltk_data'
    - 'c:\\Program Files\\Python312\\share\\nltk_data'
    - 'c:\\Program Files\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\L E G I O N\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
